In [2]:
# How to download and clean Fama French 3 factor Model data in Python
# https://www.youtube.com/watch?v=0eQv4xVbVfQ
# https://randlow.github.io/posts/finance-economics/pandas-datareader-KF/

import pandas_datareader.data as web
from pandas_datareader.famafrench import get_available_datasets
import datetime
import pickleshare
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# get the list of available datasets
datasets = get_available_datasets()
print('No. of datasets:{}'.format(len(datasets)))
#datasets

No. of datasets:297


In [4]:
datasets_factors_5F = [dataset for dataset in datasets if '_5_' in dataset and 'Factor' in dataset]
datasets_factors_3F = [dataset for dataset in datasets if '_3_' in dataset and 'Factor' in dataset]

datasets_factors_weekly = [dataset for dataset in datasets if 'weekly' in dataset and 'Factor' in dataset]


datasets_factors = [dataset for dataset in datasets if 'Factor' in dataset in dataset]
datasets_factors

['F-F_Research_Data_Factors',
 'F-F_Research_Data_Factors_weekly',
 'F-F_Research_Data_Factors_daily',
 'F-F_Research_Data_5_Factors_2x3',
 'F-F_Research_Data_5_Factors_2x3_daily',
 'F-F_Momentum_Factor',
 'F-F_Momentum_Factor_daily',
 'F-F_ST_Reversal_Factor',
 'F-F_ST_Reversal_Factor_daily',
 'F-F_LT_Reversal_Factor',
 'F-F_LT_Reversal_Factor_daily',
 'Developed_3_Factors',
 'Developed_3_Factors_Daily',
 'Developed_ex_US_3_Factors',
 'Developed_ex_US_3_Factors_Daily',
 'Europe_3_Factors',
 'Europe_3_Factors_Daily',
 'Japan_3_Factors',
 'Japan_3_Factors_Daily',
 'Asia_Pacific_ex_Japan_3_Factors',
 'Asia_Pacific_ex_Japan_3_Factors_Daily',
 'North_America_3_Factors',
 'North_America_3_Factors_Daily',
 'Developed_5_Factors',
 'Developed_5_Factors_Daily',
 'Developed_ex_US_5_Factors',
 'Developed_ex_US_5_Factors_Daily',
 'Europe_5_Factors',
 'Europe_5_Factors_Daily',
 'Japan_5_Factors',
 'Japan_5_Factors_Daily',
 'Asia_Pacific_ex_Japan_5_Factors',
 'Asia_Pacific_ex_Japan_5_Factors_Daily',

In [5]:
#create dataframe with elements of datasets_factors but not in datasets_5_factors and datasets_3_factors
datasets_factors_others = [dataset for dataset in datasets_factors if dataset not in datasets_factors_5F and dataset not in datasets_factors_3F]
datasets_factors_others

['F-F_Research_Data_Factors',
 'F-F_Research_Data_Factors_weekly',
 'F-F_Research_Data_Factors_daily',
 'F-F_Momentum_Factor',
 'F-F_Momentum_Factor_daily',
 'F-F_ST_Reversal_Factor',
 'F-F_ST_Reversal_Factor_daily',
 'F-F_LT_Reversal_Factor',
 'F-F_LT_Reversal_Factor_daily',
 'Developed_Mom_Factor',
 'Developed_Mom_Factor_Daily',
 'Developed_ex_US_Mom_Factor',
 'Developed_ex_US_Mom_Factor_Daily',
 'Europe_Mom_Factor',
 'Europe_Mom_Factor_Daily',
 'Japan_Mom_Factor',
 'Japan_Mom_Factor_Daily',
 'Asia_Pacific_ex_Japan_MOM_Factor',
 'Asia_Pacific_ex_Japan_MOM_Factor_Daily',
 'North_America_Mom_Factor',
 'North_America_Mom_Factor_Daily',
 'Emerging_MOM_Factor']

In [6]:
print(f"Number of datasets_factors: {len(datasets_factors)}")                   # 49
print(f"Number of datasets_factors_5F: {len(datasets_factors_5F)}")             # 15
print(f"Number of datasets_factors_3F: {len(datasets_factors_3F)}")             # 12
print(f"Number of datasets_factors_others: {len(datasets_factors_others)}")     # 22
print(f"Number of datasets_factors_weekly: {len(datasets_factors_weekly)}")     # 1

Number of datasets_factors: 49
Number of datasets_factors_5F: 15
Number of datasets_factors_3F: 12
Number of datasets_factors_others: 22
Number of datasets_factors_weekly: 1


In [7]:
# Fama-French 3F Model & Fama-French 5F Model

# Selecting the dataset of interest in string format
ff3_dataset_name = 'F-F_Research_Data_Factors'
ff5_dataset_name = 'F-F_Research_Data_5_Factors_2x3'
# start = date_format("2010-1-1"", %Y-%m-%d")
start = datetime.datetime(2010, 1, 1)

ff3 = web.DataReader(ff3_dataset_name, 'famafrench', start)[0]
ff5 = web.DataReader(ff5_dataset_name, 'famafrench', start)[0]

#convert in decimal format
ff3 = ff3.apply(lambda x: x/ 100)
ff5 = ff5.apply(lambda x: x/ 100)

#concatenate the two dataframes into one with subcategories in columns and add dataset name in columns header
ff3_5 = pd.concat([ff3, ff5], axis=1)

# Display Two Rows as Column Headers
ff3_name = 'Fama-French 3 Factor'
ff5_name = 'Fama-French 5 Factor'
# column headers with two lines
ff3_superheader = [ff3_name] * len(ff3.columns)
ff3_headers = list(ff3.columns)
ff3_two_lines_headers = [ff3_superheader, ff3_headers]
ff3_5.columns = ff3_two_lines_headers


#column_headers = ('Mkt-RF', 'SMB', 'HML', 'RF', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF')

ff3_5.columns = pd.MultiIndex.from_tuples([('ff3', 'Mkt-RF'), ('ff3', 'SMB'), ('ff3', 'HML'), ('ff3', 'RF'), ('ff5', 'Mkt-RF'), ('ff5', 'SMB'), ('ff5', 'HML'), ('ff5', 'RMW'), ('ff5', 'CMA'), ('ff5', 'RF')])
ff3_5


/tmp/ipykernel_85760/1208585125.py:9: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff3 = web.DataReader(ff3_dataset_name, 'famafrench', start)[0]
/tmp/ipykernel_85760/1208585125.py:9: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff3 = web.DataReader(ff3_dataset_name, 'famafrench', start)[0]
/tmp/ipykernel_85760/1208585125.py:10: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff5 = web.DataReader(ff5_dataset_name, 'famafrench', start)[0]
/tmp/ipykernel_85760/1208585125.py:10: FutureWarning: The argument 'date_parser' is deprecate

ValueError: Length mismatch: Expected axis has 10 elements, new values have 4 elements

NameError: name 'ff3_name' is not defined